In [ ]:
import os
os.environ["MUJOCO_GL"] = "egl"
    
from dataclasses import dataclass, field
from typing import Any, Callable, Dict, Tuple
from pprint import pprint

import dcargs
import glob
import hydra
import numpy as np
import torch

import torch.nn.functional as F
from omegaconf import DictConfig, OmegaConf
from torch.utils.data.dataloader import DataLoader

from research.logger import WandBLogger, WandBLoggerConfig
from research.mtm.models.mtm_model import MaskedDP, MTMConfig, make_plots_with_masks
from research.mtm.tokenizers.base import Tokenizer, TokenizerManager
import mediapy as media
from research.mtm.train import RunConfig
import matplotlib.pyplot as plt
from collections import defaultdict
from research.utils.plot_utils import PlotHandler as ph
from pathlib import Path

%matplotlib inline

In [ ]:
root = "/private/home/philippwu/mtm/outputs/mtm_mae/"

date = "2022-11-07_15-08-22" # continous
extra = "1_+experiments=exorl_continuous_rew,args.mask_patterns=[RANDOM,GOAL,ID,FD],args.model_config.n_dec_layer=6,args.model_config.n_enc_layer=6,args.model_config.n_head=4" # random masks 


path = os.path.join(root, date, extra)
path

In [ ]:
# [RANDOM,GOAL,ID,FD],args.model_config.n_dec_layer=1 
path = "/private/home/philippwu/mtm/outputs/mtm_mae/2022-11-17_18-22-57/1_+experiments=exorl_continuous_rew_qpos,args.mask_patterns=[RANDOM,GOAL,ID,FD],args.model_config.n_dec_layer=1,args.model_config.n_enc_layer=1,args.model_config.n_head=4"

In [ ]:
# [RANDOM,GOAL,ID,FD],args.model_config.n_dec_layer=2
# path = "/private/home/philippwu/mtm/outputs/mtm_mae/2022-11-17_18-23-27/1_+experiments=exorl_continuous_rew_qpos,args.mask_patterns=[RANDOM,GOAL,ID,FD],args.model_config.n_dec_layer=2,args.model_config.n_enc_layer=2,args.model_config.n_head=4"

In [ ]:
# discrete
# [RANDOM,GOAL,ID,FD],args.model_config.n_dec_layer=1
# path = "/private/home/philippwu/mtm/outputs/mtm_mae/2022-11-18_22-20-17/1_+experiments=exorl_discrete_rew_qpos,args.mask_patterns=[RANDOM,GOAL,ID,FD],args.model_config.n_dec_layer=1,args.model_config.n_enc_layer=2"

In [ ]:
# discrete
# [RANDOM,GOAL],args.model_config.n_dec_layer=1
# path = "/private/home/philippwu/mtm/outputs/mtm_mae/2022-11-18_22-20-17/4_+experiments=exorl_discrete_rew_qpos,args.mask_patterns=[RANDOM,GOAL],args.model_config.n_dec_layer=1,args.model_config.n_enc_layer=2"

In [ ]:
# discrete
# [RANDOM,GOAL],args.model_config.n_dec_layer=2
# path = "/private/home/philippwu/mtm/outputs/mtm_mae/2022-11-18_22-20-17/4_+experiments=exorl_discrete_rew_qpos,args.mask_patterns=[RANDOM,GOAL],args.model_config.n_dec_layer=2,args.model_config.n_enc_layer=2"

In [ ]:
# find checkpoints in the directory
steps = []
names = []
paths_ = os.listdir(path)
for name in [os.path.join(path, n) for n in paths_ if "pt" in n]:
    step = os.path.basename(name).split("_")[-1].split(".")[0]
    steps.append(int(step))
    names.append(name)
    print(name)

ckpt_path = names[np.argmax(steps)]

In [ ]:
# ckpt_path = '/private/home/philippwu/mtm/outputs/mtm_mae/2022-11-17_18-22-57/1_+experiments=exorl_continuous_rew_qpos,args.mask_patterns=[RANDOM,GOAL,ID,FD],args.model_config.n_dec_layer=1,args.model_config.n_enc_layer=1,args.model_config.n_head=4/model_640000.pt'

In [ ]:
torch.load(ckpt_path)["step"]

In [ ]:
hydra_cfg = OmegaConf.load(os.path.join(path, ".hydra/config.yaml"))

In [ ]:
# make datasets smaller for easier loading
hydra_cfg.dataset.train_max_size = 10000
hydra_cfg.dataset.val_max_size = 10000

In [ ]:
del hydra_cfg.args.wandb_config
cfg = hydra.utils.instantiate(hydra_cfg.args)
pprint(cfg)

In [ ]:
train_dataset, val_dataset = hydra.utils.call(
    hydra_cfg.dataset, seq_steps=cfg.model_config.traj_length
)
print("Train set size =", len(train_dataset))
print("Validation set size =", len(val_dataset))

tokenizers: Dict[str, Tokenizer] = {
    k: hydra.utils.call(v, key=k, train_dataset=train_dataset)
    for k, v in hydra_cfg.tokenizers.items()
}
tokenizer_manager = TokenizerManager(tokenizers)
discrete_map: Dict[str, bool] = {}
for k, v in tokenizers.items():
    discrete_map[k] = v.discrete
print(tokenizers)

train_loader = DataLoader(
    train_dataset,
    # shuffle=True,
    pin_memory=True,
    batch_size=cfg.batch_size,
    num_workers=cfg.n_workers,
)
val_loader = DataLoader(
    val_dataset,
    # shuffle=False,
    batch_size=cfg.batch_size,
    num_workers=cfg.n_workers,
)
train_batch = next(iter(train_loader))
tokenized = tokenizer_manager.encode(train_batch)
data_shapes = {}
for k, v in tokenized.items():
    data_shapes[k] = v.shape[-2:]
print(data_shapes)


In [ ]:
env = val_dataset._env

In [ ]:
env.reset()
image = env.physics.render(480, 640)
plt.imshow(image)

In [ ]:
sample_trajectory_with_metadata = val_dataset.sample(1)

In [ ]:
sample_trajectory_with_metadata.keys()

In [ ]:
time = 6
state = sample_trajectory_with_metadata["physics"][time]
state0 = sample_trajectory_with_metadata["physics"][0]
#env.reset()
with env.physics.reset_context():
    env.physics.set_state(state)
    
obs = env.task.get_observation(env.physics)
print(obs.keys())

In [ ]:
env.physics.get_state()
np.testing.assert_allclose(obs["orientations"], sample_trajectory_with_metadata["observations"][time][0:14])
obs["orientations"] - sample_trajectory_with_metadata["observations"][time][0:14]

In [ ]:
sample_trajectory_with_metadata.keys()

In [ ]:
print(time)
action = sample_trajectory_with_metadata["actions"][time]
new_obs = env.step(action)
new_obs["observation"] - sample_trajectory_with_metadata["observations"][time + 1]

In [ ]:
env.physics.get_state() - sample_trajectory_with_metadata["physics"][time + 1]

In [ ]:
action = sample_trajectory_with_metadata["actions"][time + 1]
new_obs = env.step(action)
new_obs["observation"] - sample_trajectory_with_metadata["observations"][time + 2]

In [ ]:
# set env physics
env.reset()
with env.physics.reset_context():
    env.physics.set_state(sample_trajectory_with_metadata["physics"][0])

# rollout actions
actions = sample_trajectory_with_metadata["actions"]
_obs = sample_trajectory_with_metadata["observations"]
images = [env.physics.render(480, 640, 0)]
for idx, action in enumerate(actions):
    obs = env.step(action)["observation"]
    image = env.physics.render(480, 640, 0)
    if idx < len(actions) - 1:
        np.testing.assert_allclose(obs, _obs[idx+1], 1e-1, 1e-1)
    images.append(image)
# media.show_video(images, fps=30)

In [ ]:
idx

In [ ]:
# set env physics
env.reset()
with env.physics.reset_context():
    env.physics.set_state(state0)

# rollout actions
actions = sample_trajectory_with_metadata["actions"]
_obs = sample_trajectory_with_metadata["observations"]
images_gt = [env.physics.render(480, 640, 0)]
for idx, p in enumerate(sample_trajectory_with_metadata["physics"]):
    env.physics.reset()
    with env.physics.reset_context():
        env.physics.set_state(p)
    image = env.physics.render(480, 640, 0)
    images_gt.append(image)

media.show_video(images_gt, fps=30)

In [ ]:
lie_back = [ -1.2 ,  0. ,  -1.57,  0, 0. , 0.0, 0, -0.,  0.0]
lie_front = [-1.2, -0, 1.57, 0, 0, 0, 0, 0., 0.]
legs_up = [ -1.24 ,  0. ,  -1.57,  1.57, 0. , 0.0,  1.57, -0.,  0.0]

kneel = [ -0.5 ,  0. ,  0,  0, -1.57, -0.8,  1.57, -1.57,  0.0]
side_angle = [ -0.3 ,  0. ,  0.9,  0, 0, -0.7,  1.87, -1.07,  0.0]
stand_up = [-0.15, 0., 0.34, 0.74, -1.34, -0., 1.1, -0.66, -0.1]

lean_back = [-0.27, 0., -0.45, 0.22, -1.5, 0.86, 0.6, -0.8, -0.4]
boat = [ -1.04 ,  0. ,  -0.8,  1.6, 0. , 0.0, 1.6, -0.,  0.0]
bridge = [-1.1, 0., -2.2, -0.3, -1.5, 0., -0.3, -0.8, -0.4]

head_stand = [-1, 0., -3, 0.6, -1, -0.3, 0.9, -0.5, 0.3]
one_feet = [-0.2, 0., 0, 0.7, -1.34, 0.5, 1.5, -0.6, 0.1]
arabesque = [-0.34, 0., 1.57, 1.57, 0, 0., 0, -0., 0.]

In [ ]:
env.physics.data.qpos = boat 

In [ ]:
env.physics.forward()
phy_state = env.physics.get_state()

In [ ]:
folder = Path("files/yoga")
folder.mkdir(parents=True, exist_ok=True)

In [ ]:
_img= env.physics.render(480, 640, 0)
media.write_image(folder / "goal.png", _img)
plt.imshow(_img)

In [ ]:
values = list(env.task.get_observation(env.physics).values())

In [ ]:
# state = np.concatenate([values[0], np.array([values[1]]), values[2]])
state = np.concatenate([values[0], np.array([values[1]])])
goal_state = state

In [ ]:
t_len = cfg.model_config.traj_length

In [ ]:
state[None].shape

In [ ]:
torch_states  = torch.from_numpy(state[None]).repeat(t_len, 1)

# Create the model

In [ ]:
model = MaskedDP(data_shapes, cfg.model_config)
model.to(cfg.device)
model.train()

# load weights
model.load_state_dict(torch.load(ckpt_path)["model"])
model.eval()
print()

In [ ]:
state.shape

In [ ]:
torch_states.shape

In [ ]:
batch_torch = {
    "states": torch_states.to(cfg.device, torch.float32).unsqueeze(0),
    "actions": torch.from_numpy(sample_trajectory_with_metadata["actions"])
    .to(cfg.device)
    .unsqueeze(0),
}


# goalreaching mask
prediction_steps = t_len-1
state_mask = torch.ones(batch_torch["states"].shape[1])
action_mask = torch.zeros(batch_torch["actions"].shape[1])
masks = {"states": state_mask, "actions": action_mask}
masks_torch = {k: v.to(cfg.device) for k, v in masks.items()}


# #shorten everything to the prediction_steps
# for k in masks.keys():
#     masks_torch[k] = masks_torch[k][:prediction_steps+1]
#     batch_torch[k] = batch_torch[k][:prediction_steps+1]

encoded_batch = tokenizer_manager.encode(batch_torch)
predicted_trajectories = model(encoded_batch, masks_torch)
decoded_trajectories = tokenizer_manager.decode(predicted_trajectories)

In [ ]:
# def get_actions():

In [ ]:
env.reset()
with env.physics.reset_context():
    env.physics.set_state(phy_state)
    
images_open_loop = [env.physics.render(480, 640, 0)]
execute_actions = decoded_trajectories["actions"].squeeze(0).detach().cpu().numpy()
traj_real_ol = defaultdict(list)
traj_real_ol["states"].append(goal_state)

In [ ]:
for idx, action in enumerate(execute_actions):
    traj_real_ol["actions"].append(action)
    obs = env.step(action)
    traj_real_ol["states"].append(obs["observation"][:15])
    image = env.physics.render(480, 640, 0)
    images_open_loop.append(image)
    
traj_real_ol["states"] = traj_real_ol["states"][:-1]
    # compare obs against data
#     _obs = sample_trajectory_with_metadata["observations"]
#     np.testing.assert_allclose(obs, _obs[idx], atol=1e-5)

In [ ]:
asdf = np.array(images_open_loop)

In [ ]:
asdf.shape

In [ ]:
media.show_video(images_open_loop, fps=30)
media.write_video(folder / "open_loop.gif", np.array(images_open_loop), codec='gif')

In [ ]:

max_n_plots = 3
for k, _ in decoded_trajectories.items():
    traj = batch_torch[k][0].detach().cpu().numpy()
    pred_traj = decoded_trajectories[k][0].detach().cpu().numpy()
    mask = masks[k]
    for i in range(min(max_n_plots, traj.shape[-1])):
        gt_i = traj[:, i]
        re_i = pred_traj[:, i]
        real_i = np.array(traj_real_ol[k])[:, i]
        if len(mask.shape) == 1:
            # only along time dimension: repeat across the given dimension
            mask = mask[:, None].repeat(1, traj.shape[1])
        select_mask = mask[:, i].cpu().numpy()
        unmasked_gt_i = gt_i[select_mask == 1]
        unmasked_gt_i_index = np.arange(len(gt_i))[select_mask == 1]
        vmax = max(np.max(gt_i), np.max(re_i), np.max(real_i))
        vmin = min(np.min(gt_i), np.min(re_i), np.min(real_i))
        y_range = vmax - vmin
        with ph.plot_context() as (fig, ax):

            ax.plot(gt_i, "-o", label="ground truth")
            ax.plot(
                re_i, "-o", label="reconstructed", markerfacecolor="none"
            )
            ax.plot(
                unmasked_gt_i_index,
                unmasked_gt_i,
                "o",
                label="unmasked ground truth",
            )
            ax.plot(
                real_i, ".", label="real"
            )
            ax.set_ylim(
                vmin - y_range / 5,
                vmax + y_range / 5,
            )
            ax.legend()
            ax.set_title(f"{k}_{i}")
            plt.show()
#             eval_logs[
#                 f"{eval_name}/batch={batch_idx}|{i}_{k}"
#             ] = wandb.Image(ph.plot_as_image(fig))


In [ ]:
# run closed loop
batch_torch = {
    "states": torch_states.to(cfg.device, torch.float32).unsqueeze(0),
    "actions": torch.from_numpy(sample_trajectory_with_metadata["actions"])
    .to(cfg.device)
    .unsqueeze(0),
}


# goalreaching mask
state_mask = torch.ones(batch_torch["states"].shape[1])
action_mask = torch.zeros(batch_torch["actions"].shape[1])
masks = {"states": state_mask, "actions": action_mask}
masks_torch = {k: v.to(cfg.device) for k, v in masks.items()}

# #shorten everything to the prediction_steps
# for k in masks.keys():
#     masks_torch[k] = masks_torch[k][:prediction_steps+1]
#     batch_torch[k] = batch_torch[k][:prediction_steps+1]

env.reset()
with env.physics.reset_context():
    env.physics.set_state(phy_state)

images_close_loop = [env.physics.render(480, 640, 0)]
traj_real_cl = defaultdict(list)
traj_real_cl["states"].append(goal_state)


for i in range(prediction_steps):
    encoded_batch = tokenizer_manager.encode(batch_torch)
    predicted_trajectories = model(encoded_batch, masks_torch)
    decoded_trajectories = tokenizer_manager.decode(predicted_trajectories)
    
    a = decoded_trajectories["actions"][0][i].detach().cpu().numpy()
    batch_torch["actions"][0][i] = torch.tensor(a, device="cuda")
    traj_real_cl["actions"].append(a)
    time_step = env.step(a)
    traj_real_cl["states"].append(time_step["observation"][:15])
    image = env.physics.render(480, 640, 0)
    images_close_loop.append(image)
    masks["states"][i] = 1
    masks["actions"][i] = 1
    batch_torch["states"][0][i] = torch.tensor(time_step["observation"][:15], device="cuda")

In [ ]:
media.show_video(images_close_loop, fps=30)
media.write_video(folder / "close_loop.gif", images_close_loop, fps=30, codec='gif')

In [ ]:
np.array(traj_real_cl[k])[0].shape

In [ ]:

max_n_plots = 3
for k, _ in decoded_trajectories.items():
    traj = torch_states.numpy()
    pred_traj = decoded_trajectories[k][0].detach().cpu().numpy()
    mask = masks[k]
    for i in range(min(max_n_plots, traj.shape[-1])):
        gt_i = traj[:, i]
        re_i = pred_traj[:, i]
        real_i = np.array(traj_real_cl[k])[:, i]
        if len(mask.shape) == 1:
            # only along time dimension: repeat across the given dimension
            mask = mask[:, None].repeat(1, traj.shape[1])
        select_mask = mask[:, i].cpu().numpy()
        unmasked_gt_i = gt_i[select_mask == 1]
        unmasked_gt_i_index = np.arange(len(gt_i))[select_mask == 1]
        vmax = max(np.max(gt_i), np.max(re_i))
        vmin = min(np.min(gt_i), np.min(re_i))
        y_range = vmax - vmin
        with ph.plot_context() as (fig, ax):

            ax.plot(gt_i, "-o", label="ground truth")
            ax.plot(
                re_i, "-o", label="reconstructed", markerfacecolor="none"
            )
            ax.plot(
                unmasked_gt_i_index,
                unmasked_gt_i,
                "o",
                label="unmasked ground truth",
            )
            ax.plot(
                real_i, ".", label="real"
            )
            ax.set_ylim(
                vmin - y_range / 5,
                vmax + y_range / 5,
            )
            ax.legend()
            ax.set_title(f"{k}_{i}")
            plt.show()
#             eval_logs[
#                 f"{eval_name}/batch={batch_idx}|{i}_{k}"
#             ] = wandb.Image(ph.plot_as_image(fig))


In [ ]:
diff_cl = goal_state - traj_real_cl["states"][-1]

In [ ]:
np.sum(diff_cl**2)

In [ ]:
diff_ol = goal_state - traj_real_ol["states"][-1][:15]

In [ ]:
np.sum(diff_ol**2)

In [ ]:
# run closed loop
batch_torch = {
    "states": torch_states.to(cfg.device, torch.float32).unsqueeze(0),
    "actions": torch.from_numpy(sample_trajectory_with_metadata["actions"])
    .to(cfg.device)
    .unsqueeze(0),
}


# goalreaching mask
state_mask = torch.zeros(batch_torch["states"].shape[1])
state_mask[:3] = 1
state_mask[-3:] = 1
action_mask = torch.zeros(batch_torch["actions"].shape[1])
masks = {"states": state_mask, "actions": action_mask}
masks_torch = {k: v.to(cfg.device) for k, v in masks.items()}

# #shorten everything to the prediction_steps
# for k in masks.keys():
#     masks_torch[k] = masks_torch[k][:prediction_steps+1]
#     batch_torch[k] = batch_torch[k][:prediction_steps+1]

env.reset()
with env.physics.reset_context():
    env.physics.set_state(phy_state)

images_close_loop = [env.physics.render(480, 640, 0)]
traj_real_cl = defaultdict(list)
traj_real_cl["states"].append(sample_trajectory_with_metadata["observations"][0][:15])

    

for i in range(prediction_steps):
    encoded_batch = tokenizer_manager.encode(batch_torch)
    predicted_trajectories = model(encoded_batch, masks_torch)
    decoded_trajectories = tokenizer_manager.decode(predicted_trajectories)
    
    a = decoded_trajectories["actions"][0][i].detach().cpu().numpy()
    batch_torch["actions"][0][i] = torch.tensor(a, device="cuda")
    traj_real_cl["actions"].append(a)
    time_step = env.step(a)
    traj_real_cl["states"].append(time_step["observation"][:15])
    image = env.physics.render(480, 640, 0)
    images_close_loop.append(image)
    masks["states"][i] = 1
    masks["actions"][i] = 1
    batch_torch["states"][0][i] = torch.tensor(time_step["observation"][:15], device="cuda")

In [ ]:
media.show_video(images_close_loop, fps=30)
media.write_video(folder / "close_loop_goal.gif", images_close_loop, fps=30, codec='gif')

In [ ]:

max_n_plots = 3
for k, _ in decoded_trajectories.items():
    traj = torch_states.numpy()
    pred_traj = decoded_trajectories[k][0].detach().cpu().numpy()
    mask = masks[k]
    for i in range(min(max_n_plots, traj.shape[-1])):
        gt_i = traj[:, i]
        re_i = pred_traj[:, i]
        real_i = np.array(traj_real_cl[k])[:, i]
        if len(mask.shape) == 1:
            # only along time dimension: repeat across the given dimension
            mask = mask[:, None].repeat(1, traj.shape[1])
        select_mask = mask[:, i].cpu().numpy()
        unmasked_gt_i = gt_i[select_mask == 1]
        unmasked_gt_i_index = np.arange(len(gt_i))[select_mask == 1]
        vmax = max(np.max(gt_i), np.max(re_i))
        vmin = min(np.min(gt_i), np.min(re_i))
        y_range = vmax - vmin
        with ph.plot_context() as (fig, ax):

            ax.plot(gt_i, "-o", label="ground truth")
            ax.plot(
                re_i, "-o", label="reconstructed", markerfacecolor="none"
            )
            ax.plot(
                unmasked_gt_i_index,
                unmasked_gt_i,
                "o",
                label="unmasked ground truth",
            )
            ax.plot(
                real_i, ".", label="real"
            )
            ax.set_ylim(
                vmin - y_range / 5,
                vmax + y_range / 5,
            )
            ax.legend()
            ax.set_title(f"{k}_{i}")
            plt.show()
#             eval_logs[
#                 f"{eval_name}/batch={batch_idx}|{i}_{k}"
#             ] = wandb.Image(ph.plot_as_image(fig))


In [ ]:
diff_cl = goal_state - traj_real_cl["states"][-1]
np.sum(diff_cl**2)

In [ ]:
goal_reach_states = torch_states.clone()
loc = 0
goal_reach_states[:3] = torch.tensor(sample_trajectory_with_metadata["observations"][loc:loc+3, :15])
physics_start = sample_trajectory_with_metadata["physics"][loc]
alpha = torch.linspace(0, 1, t_len)
goal_reach_states = goal_reach_states[0] + (goal_reach_states[-1] - goal_reach_states[0]) * alpha[:, None]


# run closed loop
batch_torch = {
    "states": goal_reach_states.to(cfg.device, torch.float32).unsqueeze(0),
    "actions": torch.from_numpy(sample_trajectory_with_metadata["actions"])
    .to(cfg.device)
    .unsqueeze(0),
}


# goalreaching mask
state_mask = torch.zeros(batch_torch["states"].shape[1])
state_mask[:3] = 1
state_mask[-3:] = 1
action_mask = torch.zeros(batch_torch["actions"].shape[1])
masks = {"states": state_mask, "actions": action_mask}
masks_torch = {k: v.to(cfg.device) for k, v in masks.items()}

encoded_batch = tokenizer_manager.encode(batch_torch)
predicted_trajectories = model(encoded_batch, masks_torch)
decoded_trajectories = tokenizer_manager.decode(predicted_trajectories)

env.reset()
with env.physics.reset_context():
    env.physics.set_state(physics_start)

images_close_loop = [env.physics.render(480, 640, 0)]
execute_actions = decoded_trajectories["actions"].squeeze(0).detach().cpu().numpy()
traj_real_cl = defaultdict(list)
traj_real_cl["states"].append(sample_trajectory_with_metadata["observations"][0][:15])



for idx, action in enumerate(execute_actions):
    traj_real_cl["actions"].append(action)
    obs = env.step(action)
    traj_real_cl["states"].append(obs["observation"][:15])
    image = env.physics.render(480, 640, 0)
    images_close_loop.append(image)
    
traj_real_cl["states"] = traj_real_cl["states"][:-1]

In [ ]:
media.show_video(images_close_loop, fps=30)
media.write_video(folder / "open_loop_reach_from_random.gif", images_close_loop, fps=30, codec='gif')

In [ ]:

max_n_plots = 3
for k, _ in decoded_trajectories.items():
    traj = goal_reach_states.numpy()
    pred_traj = decoded_trajectories[k][0].detach().cpu().numpy()
    mask = masks[k]
    for i in range(min(max_n_plots, traj.shape[-1])):
        gt_i = traj[:, i]
        re_i = pred_traj[:, i]
        real_i = np.array(traj_real_cl[k])[:, i]
        if len(mask.shape) == 1:
            # only along time dimension: repeat across the given dimension
            mask = mask[:, None].repeat(1, traj.shape[1])
        select_mask = mask[:, i].cpu().numpy()
        unmasked_gt_i = gt_i[select_mask == 1]
        unmasked_gt_i_index = np.arange(len(gt_i))[select_mask == 1]
        vmax = max(np.max(gt_i), np.max(re_i))
        vmin = min(np.min(gt_i), np.min(re_i))
        y_range = vmax - vmin
        with ph.plot_context() as (fig, ax):

            ax.plot(gt_i, "-o", label="ground truth")
            ax.plot(
                re_i, "-o", label="reconstructed", markerfacecolor="none"
            )
            ax.plot(
                unmasked_gt_i_index,
                unmasked_gt_i,
                "o",
                label="unmasked ground truth",
            )
            ax.plot(
                real_i, ".", label="real"
            )
            ax.set_ylim(
                vmin - y_range / 5,
                vmax + y_range / 5,
            )
            ax.legend()
            ax.set_title(f"{k}_{i}")
            plt.show()
#             eval_logs[
#                 f"{eval_name}/batch={batch_idx}|{i}_{k}"
#             ] = wandb.Image(ph.plot_as_image(fig))


In [ ]:
goal_reach_states = torch_states.clone()
loc = 20
goal_reach_states[:3] = torch.tensor(sample_trajectory_with_metadata["observations"][loc:loc+3, :15])
physics_start = sample_trajectory_with_metadata["physics"][loc]
alpha = torch.linspace(0, 1, t_len)
goal_reach_states = goal_reach_states[0] + (goal_reach_states[-1] - goal_reach_states[0]) * alpha[:, None]


# run closed loop
batch_torch = {
    "states": goal_reach_states.to(cfg.device, torch.float32).unsqueeze(0),
    "actions": torch.from_numpy(sample_trajectory_with_metadata["actions"])
    .to(cfg.device)
    .unsqueeze(0),
}


# goalreaching mask
state_mask = torch.zeros(batch_torch["states"].shape[1])
state_mask[:3] = 1
state_mask[-3:] = 1
action_mask = torch.zeros(batch_torch["actions"].shape[1])
masks = {"states": state_mask, "actions": action_mask}
masks_torch = {k: v.to(cfg.device) for k, v in masks.items()}

encoded_batch = tokenizer_manager.encode(batch_torch)
predicted_trajectories = model(encoded_batch, masks_torch)
decoded_trajectories = tokenizer_manager.decode(predicted_trajectories)

env.reset()
with env.physics.reset_context():
    env.physics.set_state(physics_start)

images_close_loop = [env.physics.render(480, 640, 0)]
traj_real_cl = defaultdict(list)
traj_real_cl["states"].append(sample_trajectory_with_metadata["observations"][0][:15])

for i in range(prediction_steps):
    encoded_batch = tokenizer_manager.encode(batch_torch)
    predicted_trajectories = model(encoded_batch, masks_torch)
    decoded_trajectories = tokenizer_manager.decode(predicted_trajectories)
    
    a = decoded_trajectories["actions"][0][i].detach().cpu().numpy()
    batch_torch["actions"][0][i] = torch.tensor(a, device="cuda")
    traj_real_cl["actions"].append(a)
    time_step = env.step(a)
    traj_real_cl["states"].append(time_step["observation"][:15])
    image = env.physics.render(480, 640, 0)
    images_close_loop.append(image)
    masks["states"][i] = 1
    masks["actions"][i] = 1
    batch_torch["states"][0][i] = torch.tensor(time_step["observation"][:15], device="cuda")

In [ ]:

max_n_plots = 3
for k, _ in decoded_trajectories.items():
    traj = goal_reach_states.numpy()
    pred_traj = decoded_trajectories[k][0].detach().cpu().numpy()
    mask = masks[k]
    for i in range(min(max_n_plots, traj.shape[-1])):
        gt_i = traj[:, i]
        re_i = pred_traj[:, i]
        real_i = np.array(traj_real_cl[k])[:, i]
        if len(mask.shape) == 1:
            # only along time dimension: repeat across the given dimension
            mask = mask[:, None].repeat(1, traj.shape[1])
        select_mask = mask[:, i].cpu().numpy()
        unmasked_gt_i = gt_i[select_mask == 1]
        unmasked_gt_i_index = np.arange(len(gt_i))[select_mask == 1]
        vmax = max(np.max(gt_i), np.max(re_i))
        vmin = min(np.min(gt_i), np.min(re_i))
        y_range = vmax - vmin
        with ph.plot_context() as (fig, ax):

            ax.plot(gt_i, "-o", label="ground truth")
            ax.plot(
                re_i, "-o", label="reconstructed", markerfacecolor="none"
            )
            ax.plot(
                unmasked_gt_i_index,
                unmasked_gt_i,
                "o",
                label="unmasked ground truth",
            )
            ax.plot(
                real_i, ".", label="real"
            )
            ax.set_ylim(
                vmin - y_range / 5,
                vmax + y_range / 5,
            )
            ax.legend()
            ax.set_title(f"{k}_{i}")
            plt.show()
#             eval_logs[
#                 f"{eval_name}/batch={batch_idx}|{i}_{k}"
#             ] = wandb.Image(ph.plot_as_image(fig))

In [ ]:
media.show_video(images_close_loop, fps=30)
media.write_video(folder / "closed_loop_reach_from_random.gif", images_close_loop, fps=30, codec='gif')